In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [24]:
# List of URLs to scrape
urls = [
    'https://www.screener.in/company/VOLTAS/consolidated/',
    'https://www.screener.in/company/BLUESTARCO/consolidated/',
    'https://www.screener.in/company/CROMPTON/consolidated/',
    'https://www.screener.in/company/ORIENTELEC/',
    'https://www.screener.in/company/HAVELLS/consolidated/',
    'https://www.screener.in/company/SYMPHONY/consolidated/',
    'https://www.screener.in/company/WHIRLPOOL/consolidated'
]

In [29]:
# Function to extract the required data
def extract_data_from_url(url):
    # Make a request to the webpage
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the ul element with id 'top-ratios'
    table = soup.find('ul', id='top-ratios')
    
    # Initialize empty variables for storing data
    market_cap = curr_price = high = low = stock = book_val = None
    dividend_yield = ROCE = ROE = Face_Val = None
    
    if table:
        li_elements = table.find_all('li')

        for li in li_elements:
            # Get the label and the number span
            label = li.find('span', class_='name').get_text(strip=True)
            number_span = li.find_all('span', class_='number')
            numbers = [span.get_text().replace(',', '') for span in number_span]
            
            # Assign the numbers to the correct variable based on the label
            if "Market Cap" in label:
                market_cap = float(numbers[0])
            elif "Current Price" in label:
                curr_price = float(numbers[0])
            elif "High" in label:
                high = float(numbers[0])
                low = float(numbers[1]) if len(numbers) > 1 else None
            elif "Stock P/E" in label:
                stock = float(numbers[0])
            elif "Book Value" in label:
                book_val = float(numbers[0])
            elif "Dividend Yield" in label:
                dividend_yield = float(numbers[0])
            elif "ROCE" in label:
                ROCE = float(numbers[0])
            elif "ROE" in label:
                ROE = float(numbers[0])
            elif "Face Value" in label:
                Face_Val = float(numbers[0])

    # Extract the company name from the URL (e.g., VOLTAS from 'VOLTAS')
    company_name = url.split('/')[4]
    
    # Return the extracted data as a dictionary
    return {
        'Company': company_name,
        'Market Cap': market_cap,
        'Current Price': curr_price,
        'High': high,
        'Low': low,
        'Stock P/E': stock,
        'Book Value': book_val,
        'Dividend Yield': dividend_yield,
        'ROCE': ROCE,
        'ROE': ROE,
        'Face Value': Face_Val
    }

# List to hold all the company data
all_data = []

# Loop through each URL and scrape the data
for url in urls:
    company_data = extract_data_from_url(url)
    all_data.append(company_data)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(all_data)

In [30]:
df

,Company,Market Cap,Current Price,High,Low,Stock P/E,Book Value,Dividend Yield,ROCE,ROE,Face Value
0,VOLTAS,61766.0,1867.0,1946.0,807.0,136.0,176.0,0.29,8.51,4.40,1.0
1,BLUESTARCO,43580.0,2120.0,2198.0,814.0,87.2,127.0,0.33,25.50,21.20,2.0
2,CROMPTON,27213.0,423.0,484.0,261.0,58.2,46.6,0.71,15.80,15.60,2.0
3,ORIENTELEC,5104.0,239.0,297.0,189.0,92.5,30.0,0.63,14.20,10.30,1.0
4,HAVELLS,127346.0,2031.0,2106.0,1233.0,91.5,119.0,0.44,24.40,18.00,1.0
5,SYMPHONY,11150.0,1617.0,1749.0,820.0,52.6,109.0,0.80,18.80,18.00,2.0
6,WHIRLPOOL,27930.0,2201.0,2281.0,1186.0,111.0,254.0,0.23,8.34,5.69,10.0


In [31]:
df.to_csv("Basic_Pokemon_Stats.csv",index=False)